In [ ]:
# Importing essential libraries
from flask import Flask, render_template, request, redirect, url_for, flash
import logging, re, os
from datetime import datetime


# ------------------------------------
#Initialize Flask Application
app = Flask(__name__)
# ------------------------------------


# ------------------------------------
# In production set SECRET_KEY via environment variable
app.secret_key = os.environ.get("SECRET_KEY", "dev-change-me")
# ------------------------------------


# ------------------------------------
#Configure Logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[logging.FileHandler("app.log"), logging.StreamHandler()],
)
log = logging.getLogger(__name__)
# ------------------------------------


# ------------------------------------
# Simple in-memory storage
REGISTERED_USERS = [] # each item: {"username", "email", "age", "created_at"}
# ------------------------------------


# ------------------------------------
# Validation patterns

##Ensure that first name is only letters and hyphens
FIRSTNAME_PATTERN = re.compile(r'^[A-Za-z-]{1,50}$')

##Ensure that last name is only letters and hyphens
LASTNAME_PATTERN = re.compile(r'[A-Za-z-]{1,50}$')

## Ensures the email has a basic valid structure of name@domain.tld
EMAIL_PATTERN = re.compile(r'^[\w\.-]+@[\w\.-]+\.[A-Za-z]{2,}$')

## Ensures the username only allows letters, digits, underscores, and length between 5 and 16 characters.
USERNAME_PATTERN = re.compile(r'^[A-Za-z0-9_]{5,16}$')

## Strong password pattern that requires lowercase, uppercase, digit, special character, and minimum 8 characters.
PASSWORD_PATTERN = re.compile(r'^(?=.*[a-z])(?=.*[A-Z])(?=.*\d)(?=.*[@$!%*?&]).{8,}$')
# ------------------------------------


# ------------------------------------
# Route for the Home page
@app.route('/')
def home():
    return render_template('Home_Page.html')

# Route for the About page
@app.route('/about')
def about():
    return render_template('About.html')

# Route for Registration success page 
@app.route('/success')
def success():
    return render_template('Registration_Success.html')

# Route for User Registration page
## Simple register form that intakes username, email and password for now
##To test that validation patterns are working 
@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        First_name = (request.form.get("First_name") or "").strip()
        Last_name = (request.form.get("Last_name") or "").strip()
        username = (request.form.get("username") or "").strip()
        email    = (request.form.get("email") or "").strip()
        password = (request.form.get("password") or "").strip()
        
        try:
             
            # ----- Checking empty fields -----
             if not First_name:
                 raise ValueError("First name is required")
             if not Last_name:
                 raise ValueError("Last name is required")
             if not username:
                 raise ValueError("Username is required")
             if not email:
                 raise ValueError("Email is required.")
             if not password:
                 raise ValueError("Password is required.")
             
            # ----- Type / Format checks -----
             if not FIRSTNAME_PATTERN.fullmatch(First_name):
                  raise ValueError("First name must only contain letters and hyphens")

             if not LASTNAME_PATTERN.fullmatch(Last_name):
                  raise ValueError("Last name must only contain letters and hyphens")
             
             if not USERNAME_PATTERN.fullmatch(username):
                  raise ValueError("Username must be 5–16 chars (letters, digits, underscore).")
             
             if not EMAIL_PATTERN.fullmatch(email):
                  raise ValueError("Email format is invalid.")
             
             if not PASSWORD_PATTERN.fullmatch(password):
                  raise ValueError("Password format is invalid.")
             
             if len(email) > 254:
                  raise ValueError("Email too long.")

            # ----- Business rules / Whitelist checks -----
             if username.lower() in {"admin", "root"}:
                 raise ValueError("Username is reserved.")
        
            # ----- Save the user -----
             REGISTERED_USERS.append({
                 "First_name": First_name,
                 "Last_name": Last_name,
                 "username": username,
                 "email": email,
                 "password": password,
                 "created_at": datetime.utcnow()
             })
             
             flash(f"✅ Registered {username} successfully!", "success")
             log.info("Registration OK: First_name=%s Last_name=%s username=%s email=%s password=%s", First_name, Last_name, username, email, password)
             return redirect(url_for("success"))
        
        except ValueError as e:
            flash(str(e), 'error')
            log.warning("Validation failed: %s", e)
            return redirect(url_for("Registration_Form.html"))
        
    return render_template("Registration_Form.html")

# Route for Login Display page
@app.route('/login')
def login():
    user_ip = request.remote_addr  # capture visitor IP
    app.logger.info(f"Contact page accessed from IP: {user_ip}")
    return render_template('Login.html')

@app.errorhandler(404)
def not_found(e):
    # templates/404.html uses: {{ url_for('static', filename='images/3.jpg') }}
    log.warning("404 Not Found: %s", request.path)
    return render_template("404.html"), 404

if __name__ == "__main__":
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


2025-11-06 15:31:22,303 [INFO] WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
2025-11-06 15:31:22,308 [INFO] Press CTRL+C to quit
